In [1]:
# 提高输出效率库
from IPython.core.interactiveshell import InteractiveShell # 实现 notebook 的多行输出
InteractiveShell.ast_node_interactivity = 'all' #默认为'last'

import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('customers_data.csv')
df['付款时间'] = pd.to_datetime(df['付款时间'])
df['付款年月'] = df['付款时间'].dt.strftime('%Y-%m')
df.sample(10)

,脱敏客户ID,付款时间,支付金额,付款年月
13311,cumid11500,2023-11-11 15:47:47,290.14,2023-11
13433,cumid67,2023-11-11 19:34:15,310.81,2023-11
9040,cumid8186,2023-10-25 09:22:31,49.86,2023-10
32439,cumid25481,2024-02-06 18:37:13,690.35,2024-02
27401,cumid16389,2024-01-14 12:39:30,17.45,2024-01
5586,cumid5149,2023-10-17 12:11:18,9.34,2023-10
15885,cumid13561,2023-11-29 20:26:27,23.61,2023-11
31145,cumid24505,2024-02-03 13:01:32,17.78,2024-02
31387,cumid24689,2024-02-03 23:09:38,16.92,2024-02
31158,cumid24516,2024-02-03 13:33:52,17.16,2024-02


In [8]:
month_list = df['付款年月'].unique()

In [14]:
# 存储每月新增的客户数
monthly_increase = {} # 用字典存储，格式输出漂亮些，也有助于添加进 DataFrame 中

for i in range(0, len(month_list)):
    
    # 筛选出 month_list 中的每月消费，并统计客户数量
    print(f'下面统计: {month_list[i]} 的新增情况...')
    current_data = df[ df['付款年月']==month_list[i] ]
    current_clients = current_data['脱敏客户ID'].unique()
    
    # =========================== 统计新增情况 ==================================
    # 跳过数据集中的第一个月，因为没有历史数据来验证该客户是否为新增客户
    if i == 0:
        print(f'{month_list[i]} 是第一个月，无需验证客户是否为新增客户。')
        new_clients_num = len(current_clients)
        print(f'该月的新增用户数为：{new_clients_num}')
        monthly_increase[month_list[i]] = new_clients_num
        
    else:
        # 筛选该月（current_month）之前的所有历史消费记录
        history_month = month_list[:i]
        print(f'{month_list[i]} 的历史年月为：{history_month}')
        history_data = df[ df['付款年月'].isin(history_month) ]
        # 筛选未在历史消费记录中出现过的新增客户
        new_users = np.setdiff1d(current_data['脱敏客户ID'], history_data['脱敏客户ID'])
        print(f'相较于历史年月，该月的新增客户数为：{len(new_users)}')
        monthly_increase[month_list[i]] = len(new_users)
        
    # =========================== 统计客户平均消费情况 ==================================
    print('-'*50)
    print('下面统计该月之后的每个月的客户平均消费金额情况...')
    for j in range(i+1, len(month_list)):
        next_month_data = df[ df['付款年月']==month_list[j] ]
        # 统计各群的用户平均消费金额
        user_purchase = next_month_data.groupby('脱敏客户ID')['支付金额'].agg('mean')
        # 统计既出现在该月，又出现在下个月的用户
        next_month_retain = np.intersect1d(current_data['脱敏客户ID'], next_month_data['脱敏客户ID'])
        print(f'{month_list[j]} 的留存人数：{len(next_month_retain)}，客户平均消费金额：{round(user_purchase.mean())}')
    print('\n')

下面统计: 2023-09 的新增情况...
2023-09 是第一个月，无需验证客户是否为新增客户。
该月的新增用户数为：2031
--------------------------------------------------
下面统计该月之后的每个月的客户平均消费金额情况...
2023-10 的留存人数：252，客户平均消费金额：38
2023-11 的留存人数：216，客户平均消费金额：101
2023-12 的留存人数：163，客户平均消费金额：97
2024-01 的留存人数：156，客户平均消费金额：105
2024-02 的留存人数：164，客户平均消费金额：124


下面统计: 2023-10 的新增情况...
2023-10 的历史年月为：['2023-09']
相较于历史年月，该月的新增客户数为：7043
--------------------------------------------------
下面统计该月之后的每个月的客户平均消费金额情况...
2023-11 的留存人数：623，客户平均消费金额：101
2023-12 的留存人数：491，客户平均消费金额：97
2024-01 的留存人数：488，客户平均消费金额：105
2024-02 的留存人数：491，客户平均消费金额：124


下面统计: 2023-11 的新增情况...
2023-11 的历史年月为：['2023-09' '2023-10']
相较于历史年月，该月的新增客户数为：4732
--------------------------------------------------
下面统计该月之后的每个月的客户平均消费金额情况...
2023-12 的留存人数：637，客户平均消费金额：97
2024-01 的留存人数：562，客户平均消费金额：105
2024-02 的留存人数：486，客户平均消费金额：124


下面统计: 2023-12 的新增情况...
2023-12 的历史年月为：['2023-09' '2023-10' '2023-11']
相较于历史年月，该月的新增客户数为：4979
--------------------------------------------------
下面统计该月之后的每个月的客户平均消费金额情况..

In [15]:
monthly_increase
# DataFrame 的索引
result_index = list(monthly_increase.keys()) 
# 当月新增
current_month_increase = list(monthly_increase.values())

result_index
current_month_increase

{'2023-09': 2031,
 '2023-10': 7043,
 '2023-11': 4732,
 '2023-12': 4979,
 '2024-01': 5110,
 '2024-02': 7101}

['2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02']

[2031, 7043, 4732, 4979, 5110, 7101]

In [20]:
plus_one = [38, 101, 97, 105, 124, 0]
plus_two = [101, 97, 105, 124, 0, 0]
plus_three = [97, 105, 124, 0, 0, 0]
plus_four = [105, 124, 0, 0, 0, 0]
plus_five = [124, 0, 0, 0, 0, 0]

data = {'当月新增': current_month_increase,
       '+1月': plus_one, '+2月': plus_two, 
       '+3月': plus_three, '+4月': plus_four, '+5月': plus_five}

result = pd.DataFrame(data, index=result_index, columns=list(data.keys()))
result.replace(0, '-', inplace=True)
result

,当月新增,+1月,+2月,+3月,+4月,+5月
2023-09,2031,38,101,97,105,124
2023-10,7043,101,97,105,124,-
2023-11,4732,97,105,124,-,-
2023-12,4979,105,124,-,-,-
2024-01,5110,124,-,-,-,-
2024-02,7101,-,-,-,-,-


In [21]:
result.to_clipboard()